## 🔗 Step 5: Add MCP (Model Context Protocol) Integration

Now let's add MCP functionality to connect your Ollama model with Snowflake data!


In [ ]:
# Step 5.1: Install Required Packages for MCP
print("📦 Installing MCP and Snowflake packages...")

%pip install snowflake-connector-python==3.0.0 python-dotenv

print("✅ Packages installed!")


In [ ]:
# Step 5.2: Setup Snowflake Credentials
import os
import snowflake.connector
from datetime import datetime
import json

def setup_snowflake_credentials():
    """Setup Snowflake credentials in Colab"""
    print("🔐 Snowflake Credentials Setup")
    print("=" * 40)
    
    # Option A: Enter credentials interactively (recommended for security)
    use_interactive = input("Enter credentials interactively? (y/n): ").lower() == 'y'
    
    if use_interactive:
        os.environ['SNOWFLAKE_USER'] = input("Snowflake User: ")
        os.environ['SNOWFLAKE_PASSWORD'] = input("Snowflake Password: ")
        os.environ['SNOWFLAKE_ACCOUNT'] = input("Snowflake Account: ")
        os.environ['SNOWFLAKE_DATABASE'] = input("Snowflake Database (optional): ") or "SNOWFLAKE_SAMPLE_DATA"
        os.environ['SNOWFLAKE_WAREHOUSE'] = input("Snowflake Warehouse: ") or "COMPUTE_WH"
        print("✅ Credentials set interactively")
    else:
        # Option B: Use pre-configured credentials (update these with your values)
        print("💡 Using pre-configured credentials (update the values below):")
        os.environ['SNOWFLAKE_USER'] = 'ENTER YOUR CREDS HERE'
        os.environ['SNOWFLAKE_PASSWORD'] = 'ENTER YOUR CREDS HERE'  
        os.environ['SNOWFLAKE_ACCOUNT'] = 'ENTER YOUR CREDS HERE'
        os.environ['SNOWFLAKE_DATABASE'] = 'ENTER YOUR CREDS HERE'
        os.environ['SNOWFLAKE_WAREHOUSE'] = 'ENTER YOUR CREDS HERE'
        print("✅ Using predefined credentials")
    
    return True

# Setup credentials
setup_snowflake_credentials()


In [ ]:
# Step 5.3: Create Embedded MCP Server
class EmbeddedMCPServer:
    """Embedded MCP server that works directly in Colab"""
    
    def __init__(self):
        self.connection = None
        
    def connect_snowflake(self):
        """Connect to Snowflake"""
        try:
            print("🔌 Connecting to Snowflake...")
            self.connection = snowflake.connector.connect(
                user=os.environ.get('SNOWFLAKE_USER'),
                password=os.environ.get('SNOWFLAKE_PASSWORD'),
                account=os.environ.get('SNOWFLAKE_ACCOUNT'),
                database=os.environ.get('SNOWFLAKE_DATABASE'),
                warehouse=os.environ.get('SNOWFLAKE_WAREHOUSE'),
                insecure_mode=True  # Bypass SSL issues in Colab
            )
            print("✅ Connected to Snowflake successfully!")
            
            # Test the connection
            cursor = self.connection.cursor()
            cursor.execute("SELECT CURRENT_VERSION() as version, CURRENT_USER() as user")
            result = cursor.fetchone()
            print(f"📊 Snowflake Version: {result[0]}")
            print(f"👤 Connected as: {result[1]}")
            cursor.close()
            
            return True
        except Exception as e:
            print(f"❌ Snowflake connection failed: {e}")
            return False
    
    def execute_query(self, query):
        """Execute Snowflake query and return results"""
        if not self.connection:
            return {"error": "Not connected to Snowflake. Run connect_snowflake() first."}
            
        try:
            cursor = self.connection.cursor()
            cursor.execute(query)
            
            # Get column names
            columns = [desc[0] for desc in cursor.description] if cursor.description else []
            
            # Fetch results
            results = cursor.fetchall()
            
            # Convert to list of dictionaries
            data = []
            for row in results:
                row_dict = {}
                for i, value in enumerate(row):
                    if i < len(columns):
                        # Handle datetime objects
                        if hasattr(value, 'isoformat'):
                            row_dict[columns[i]] = value.isoformat()
                        else:
                            row_dict[columns[i]] = value
                data.append(row_dict)
            
            cursor.close()
            
            return {
                "success": True,
                "columns": columns,
                "data": data,
                "row_count": len(data),
                "execution_time": datetime.now().isoformat()
            }
            
        except Exception as e:
            return {
                "error": str(e),
                "query": query
            }
    
    def get_sample_tables(self):
        """Get list of available sample tables"""
        sample_queries = {
            "customer_data": "SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER LIMIT 5",
            "orders_data": "SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS LIMIT 5", 
            "sales_data": "SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.STORE_SALES LIMIT 5"
        }
        return sample_queries
    
    def close(self):
        """Close Snowflake connection"""
        if self.connection:
            self.connection.close()
            print("🔌 Snowflake connection closed")

# Initialize embedded MCP server
print("🏠 Initializing Embedded MCP Server...")
mcp_server = EmbeddedMCPServer()

# Connect to Snowflake
mcp_server.connect_snowflake()


In [ ]:
# Step 5.4: Create Enhanced Ollama Client with MCP Integration
class OllamaWithMCP:
    """Enhanced Ollama client that combines AI with Snowflake data access"""
    
    def __init__(self, mcp_server):
        self.mcp = mcp_server
        self.conversation_history = []
        
    def generate_sql_with_ai(self, user_question, model_name="llama2:7b"):
        """Use AI to generate SQL queries based on user questions"""
        
        sql_prompt = f"""You are a SQL expert. Generate a Snowflake SQL query for this question: {user_question}

Available sample databases and tables:
- SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER (customer information)
- SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS (order data)
- SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.LINEITEM (order line items)
- SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.STORE_SALES (retail sales data)

Rules:
1. Return ONLY the SQL query, no explanation
2. Use LIMIT 10 for safety unless user asks for more
3. Use proper Snowflake syntax
4. Focus on the TPCH_SF1 or TPCDS_SF10TCL schemas

Question: {user_question}

SQL Query:"""

        print(f"🤖 Generating SQL with {model_name}...")
        
        # Use subprocess to call ollama
        import subprocess
        try:
            result = subprocess.run([
                'ollama', 'run', model_name, sql_prompt
            ], capture_output=True, text=True, timeout=30)
            
            if result.returncode == 0:
                sql_query = result.stdout.strip()
                # Clean up the SQL
                sql_query = sql_query.replace('```sql', '').replace('```', '').strip()
                return sql_query
            else:
                return f"Error generating SQL: {result.stderr}"
                
        except subprocess.TimeoutExpired:
            return "Error: SQL generation timed out"
        except Exception as e:
            return f"Error calling Ollama: {str(e)}"
    
    def execute_ai_query(self, user_question, model_name="llama2:7b"):
        """Generate SQL with AI and execute it on Snowflake"""
        
        print(f"📝 Question: {user_question}")
        print("=" * 60)
        
        # Step 1: Generate SQL with AI
        sql_query = self.generate_sql_with_ai(user_question, model_name)
        
        if sql_query.startswith("Error"):
            return {"error": sql_query}
        
        print(f"🔍 Generated SQL:")
        print(f"```sql\n{sql_query}\n```")
        print()
        
        # Step 2: Execute the SQL
        print("⚡ Executing query...")
        result = self.mcp.execute_query(sql_query)
        
        if "error" in result:
            print(f"❌ Query failed: {result['error']}")
            return result
        
        # Step 3: Display results
        print(f"✅ Query successful! Found {result['row_count']} rows")
        print("\n📊 Results:")
        
        if result['data']:
            # Display first few rows nicely
            for i, row in enumerate(result['data'][:5]):
                print(f"\nRow {i+1}:")
                for key, value in row.items():
                    print(f"  {key}: {value}")
            
            if result['row_count'] > 5:
                print(f"\n... and {result['row_count'] - 5} more rows")
        
        return {
            "question": user_question,
            "sql": sql_query,
            "results": result,
            "success": True
        }
    
    def chat_with_data(self, user_message, model_name="llama2:7b"):
        """Interactive chat that can access your Snowflake data"""
        
        # Check if the user is asking about data
        data_keywords = ['show', 'find', 'get', 'select', 'data', 'table', 'customer', 'order', 'sales']
        
        if any(keyword in user_message.lower() for keyword in data_keywords):
            print("🔍 Detected data query - using AI + Snowflake...")
            return self.execute_ai_query(user_message, model_name)
        else:
            print("💭 General question - using AI only...")
            # Use subprocess for general questions
            import subprocess
            try:
                result = subprocess.run([
                    'ollama', 'run', model_name, user_message
                ], capture_output=True, text=True, timeout=30)
                
                if result.returncode == 0:
                    response = result.stdout.strip()
                    print(f"🤖 Response: {response}")
                    return {"response": response, "type": "general"}
                else:
                    return {"error": f"Ollama error: {result.stderr}"}
                    
            except Exception as e:
                return {"error": f"Error: {str(e)}"}

# Initialize the enhanced Ollama client
print("🚀 Creating Enhanced Ollama + MCP Client...")
ai_with_data = OllamaWithMCP(mcp_server)
print("✅ Ready for AI-powered data queries!")


In [ ]:
# Test 1: Basic Data Query
print("🧪 Test 1: AI-Generated Data Query")
print("=" * 50)

# Ask the AI to generate and execute a data query
test_question = "show me data from any table"

result = ai_with_data.execute_ai_query(test_question, model_name)

if result.get("success"):
    print("\n🎉 Success! AI generated and executed the query!")
else:
    print(f"\n❌ Test failed: {result.get('error', 'Unknown error')}")

print("\n" + "=" * 50)
